## Imports

In [1]:
import sys
# This is a workaround for running the script from the parent directory
sys.path.append('../')

# For printing
import pprint

In [2]:
from src.logic.load_models import load_gpt_model, load_web_searcher
from src.logic.utils import load_json, load_sub_data
from src.logic.context import generate_character_list, generate_high_level_summary, generate_web_context

## Set up required items

In [3]:
json_data = load_json("../config.json")
openai_api_key = json_data['openai_api_key']
tavily_api_key = json_data["tavily_api_key"]

llm = load_gpt_model(api_key=openai_api_key, temperature=0.3)
tavily = load_web_searcher(api_key=tavily_api_key)

In [4]:
sub_data = load_sub_data("../sample/sample_sub_gakumas.ass")
pprint.pp(sub_data[:10])

['プロデューサー: 気になっている子だが…\\N少し、話をしに行ってみるか。',
 'リーリヤ: …うぅ…',
 'リーリヤ: ど、どうしよう…',
 'プロデューサー: お困りですか？',
 'リーリヤ: えっ？あっ！',
 'リーリヤ: あの…その…ええと…',
 'プロデューサー: 失礼しました。俺はウツミ。\\Nプロデューサー科の生徒です。',
 'リーリヤ: か、葛城リーリヤ、です。',
 'リーリヤ: 驚いてしまって…ごめんなさい。',
 'プロデューサー: いえ、突然話しかけた俺が悪いんです。\\Nお困りの様子だったので、つい。']


In [5]:
transcript = "\n".join(sub_data)
pprint.pp(transcript[:200])

('プロデューサー: 気になっている子だが…\\N少し、話をしに行ってみるか。\n'
 'リーリヤ: …うぅ…\n'
 'リーリヤ: ど、どうしよう…\n'
 'プロデューサー: お困りですか？\n'
 'リーリヤ: えっ？あっ！\n'
 'リーリヤ: あの…その…ええと…\n'
 'プロデューサー: 失礼しました。俺はウツミ。\\Nプロデューサー科の生徒です。\n'
 'リーリヤ: か、葛城リーリヤ、です。\n'
 'リーリヤ: 驚いてしまって…ごめんなさい。\n'
 'プロデューサー: い')


## Testing

In [6]:
web_context = generate_web_context(llm, tavily, 
                                   input_lang="ja", output_lang='en', 
                                   series_name="Gakuen Idolmaster", keywords="sumika, lilja")
pprint.pp(web_context)

('"Gakuen Idolmaster" is a fictional work centered around the lives of '
 'aspiring idols at Hatsuboshi Gakuen. Two significant characters in this '
 'story are Lilja Katsuragi and Sumika Shiun. Lilja is a new student from '
 'overseas who, despite lacking experience in singing or dancing, is deeply '
 'passionate about becoming an idol. Her determination is fueled by a promise '
 'she made with her best friend, Sumika, to stand on stage together. Lilja is '
 'characterized by her hard work and genuine admiration for idols, although '
 'she often struggles with confidence.\n'
 '\n'
 'Sumika Shiun, on the other hand, is a lively and energetic character known '
 'for her carefree attitude. Despite her playful nature and tendency to skip '
 'classes, she is a supportive friend to Lilja. Sumika has a background in '
 'ballet but lacks the motivation to pursue it seriously. Her decision to '
 'enroll in Hatsuboshi Gakuen was largely influenced by her promise to Lilja, '
 'as she herself doe

In [7]:
character_list = generate_character_list(llm, 'ja', 'en', transcript, web_context)
pprint.pp(character_list)

('- **Lilja Katsuragi**: A new student from overseas aspiring to become an '
 'idol. She lacks experience in singing and dancing but is deeply passionate '
 'and determined, driven by a promise made with her best friend. Lilja often '
 'struggles with confidence but shows strong will and perseverance. [Narrative '
 'Focus]\n'
 '\n'
 '- **Sumika Shiun** (also referred to as "清夏ちゃん"): Lilja\'s best friend, '
 'known for her lively and carefree attitude. She has a background in ballet '
 'but lacks motivation for idol activities. Sumika is supportive of Lilja and '
 'influences her decision to enroll in the idol school.\n'
 '\n'
 '- **Producer** (also referred to as "センパイ" or "ウツミ"): A student from the '
 'producer course who supports and guides Lilja in her journey to become an '
 'idol. He is knowledgeable, encouraging, and plays a mentor role for Lilja.\n'
 '\n'
 '- **Idol Course Student A**: A student who interacts with Lilja and comments '
 'on her situation. Provides feedback and en

In [7]:
high_level_summary = generate_high_level_summary(llm, 'ja', 'en', transcript, character_list)
pprint.pp(high_level_summary)

NameError: name 'character_list' is not defined

## Web Search

In [ ]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from src.logic.context import generate_character_reading

In [7]:
import os
os.environ["TAVILY_API_KEY"] = json_data["tavily_api_key"]
search_tool = TavilySearchResults(k=3)

In [ ]:
web_search_results = generate_character_reading(llm, search_tool, output_lang="en",
                                             series_name="destiny 2", keywords="episode heresy", transcript=lines)

pprint.pp(web_search_results)

('**Character Relationships:** The story involves three major Hive Gods: Oryx, '
 'Xivu Arath, and Savathun, who are central to the narrative. Eris is another '
 'key character whose actions against Oryx lead to significant conflict, and '
 'there is a potential alliance forming between Xivu Arath and Savathun.\n'
 '\n'
 '**Story Premise and Genre:** Destiny 2 is set in a science fiction universe '
 'where players navigate through complex narratives involving gods and '
 'mythical entities. The story in the Heresy acts focuses on themes of '
 'betrayal, power struggles, and the uncovering of hidden truths about The '
 'Hive and their connection to The Light.')


## Character Identification

In [ ]:
from src.logic.context import generate_character_list

In [ ]:
characters_output = generate_character_list(llm, input_lang="ja", output_lang="en",
                                        transcript=lines, format_description=format_results, web_context=web_search_results)

pprint.pp(characters_output)

('- **誘導者の残響**: The Echo of the Inducer, a resurrected dark god from the '
 'abyss, referred to as the highest inducer. Engages in discussions about '
 'power and identity. [Narrative Focus]\n'
 '- **スロアン**: A character expressing disbelief and urgency, suggesting a sense '
 'of urgency and concern.\n'
 '- **シヴ・アラス**: Refers to the Echo of the Inducer as a brother and is deeply '
 'involved in the dialogue about betrayal and power struggles. Shows loyalty '
 'and determination.\n'
 '- **サバスン**: Engages in conversations about truth and deception, challenging '
 'the beliefs and actions of others. Appears strategic and manipulative.\n'
 '- **エリス・モーン**: Provides commentary on the situation, pointing out flaws and '
 'offering a perspective on the unfolding events. Seems to hold a position of '
 'insight or observation.')


## Summarize

In [ ]:
from src.logic.context import generate_high_level_summary

In [ ]:
summary_output = generate_high_level_summary(llm, input_lang="ja", output_lang="en",
                                 transcript=lines, format_description=format_results, 
                                 web_context=web_search_results, character_list=characters_output)

pprint.pp(summary_output)

('In this scene, the Echo of the Inducer, a resurrected dark god, confronts '
 'several characters, including Sloane, Xivu Arath, Savathun, and Eris Morn, '
 'in a tense and dynamic exchange. The Echo grapples with its identity and the '
 'implications of its resurrection, questioning its own existence and the '
 'betrayal it feels. Xivu Arath shows loyalty to the Echo, urging it to '
 "reclaim its throne through vengeance, while Savathun challenges the Echo's "
 'beliefs, revealing that the gods were manipulated by a greater force. '
 "Savathun's manipulative nature is evident as she tries to sway the Echo and "
 'others to her perspective, suggesting a path of independence from divine '
 'influences. Eris Morn provides commentary on the unfolding events, '
 "highlighting the flaws in Oryx's logic and the resulting chaos. The scene is "
 'filled with emotional tension, as characters confront their past, their '
 'loyalties, and the shifting power dynamics among them. Ultimately, the E

## Tone

In [13]:
from src.logic.context import determine_tone

In [14]:
tone_output = determine_tone(llm, input_lang='ja', output_lang='en',
                             transcript=lines, format_description=format_results,
                             web_context=web_search_results)

pprint.pp(tone_output)

('The overall tone of the scene is serious and intense, with a focus on themes '
 'of power, betrayal, and existential reflection. The speech is formal and '
 'direct, with characters often using commanding and assertive language, '
 'reflecting their high stakes and complex relationships.')
